In [32]:
import pandas as pd

# Read the CSV dataset into a pandas DataFrame
df = pd.read_csv('Book4.csv')

# Identify the dependent variable column
dependent_var = 'Highest Injury Severity'


In [2]:
# Drop rows with missing values in the dependent variable column
df = df.dropna(subset=[dependent_var])


In [3]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Rear Left" column
df['SVc_RL'] = df['SV Contact Area - Rear Left'].replace({'Y': 1, '': 0})


In [4]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Rear" column
df['SVc_R'] = df['SV Contact Area - Rear'].replace({'Y': 1, '': 0})

In [5]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Rear Right" column
df['SVc_RR'] = df['SV Contact Area - Rear Right'].replace({'Y': 1, '': 0})

In [6]:
# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Front Left" column
df['SVc_FL'] = df['SV Contact Area - Front Left'].replace({'Y': 1, '': 0})

# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Front" column
df['SVc_F'] = df['SV Contact Area - Front'].replace({'Y': 1, '': 0})

# Replace 'Y' with 1 and blanks with 0 in "SV Contact Area - Front Right" column
df['SVc_FR'] = df['SV Contact Area - Front Right'].replace({'Y': 1, '': 0})

In [7]:
# Replace 'Y' cells with 1
df['SV Contact Area - Left'] = df['SV Contact Area - Left'].replace('Y', 1)
df['SV Contact Area - Right'] = df['SV Contact Area - Right'].replace('Y', 1)


# Replace blank values with 0
df['SV Contact Area - Left'] = df['SV Contact Area - Left'].replace(' ', 0)
df['SV Contact Area - Right'] = df['SV Contact Area - Right'].replace(' ', 0)

# Convert the columns to numeric
df['SV Contact Area - Left'] = pd.to_numeric(df['SV Contact Area - Left'])
df['SV Contact Area - Right'] = pd.to_numeric(df['SV Contact Area - Right'])

# Create a new variable based on the summation of "SV Contact Area - Left" and "SV Contact Area - Right"
def new_var(row):
    if row['SV Contact Area - Left'] + row['SV Contact Area - Right'] >= 1:
        return 1
    else:
        return 0

df['SVc_Sides'] = df.apply(new_var, axis=1)


In [8]:
def weather_condition(row):
    if row['Weather - Clear'] == 'Y':
        return 0
    elif row['Weather - Cloudy'] == 'Y':
        return 1
    elif row['Weather - Rain'] == 'Y':
        return 2
    else:
        return 0

df['Weather'] = df.apply(weather_condition, axis=1)
df['Weather'] = df['Weather'].fillna(0).astype(int)


In [9]:
def lighting_to_numeric(row):
    if row['Lighting'] == 'Daylight' or row['Lighting'] == 'Dawn / Dusk':
        return 0
    elif row['Lighting'] == 'Dark - Lighted' or row['Lighting'] == 'Dark - Unknown Lighting':
        return 1
    elif row['Lighting'] == 'Dark - Not Lighted':
        return 2
    else:
        return 99

df['Lighting_recode'] = df.apply(lighting_to_numeric, axis=1)


In [10]:
def new_var(row):
    if row['SV Pre-Crash Movement'] == "Proceeding Straight":
        return 0
    elif row['SV Pre-Crash Movement'] == "Stopped":
        return 1
    elif row['SV Pre-Crash Movement'] in ["Changing Lanes", "Lane / Road Departure"]:
        return 2
    elif row['SV Pre-Crash Movement'] in ["Making Right Turn", "Making Left Turn"]:
        return 3
    elif row['SV Pre-Crash Movement'] in ["Backing", "Other, see Narrative", "Making U-Turn", "Traveling Wrong Way", "Merging", "Unknown", "Parking Maneuver"]:
        return 4
    else:
        return 99

# Apply the function to create a new variable
df['SV_Pre_Crash_M'] = df.apply(new_var, axis=1)

# Replace blanks with 99
df['SV_Pre_Crash_M'].replace(" ", 99, inplace=True)


In [25]:
def new_var(row):
    if row['Roadway Type'] == "Highway / Freeway":
        return 1
    elif row['Roadway Type'] == "Intersection":
        return 2
    elif row['Roadway Type'] == "Street":
        return 3
    elif row['Roadway Type'] == "Parking Lot":
        return 4
    elif row['Roadway Type'] == "Unknown" or row['Roadway Type'] == "Traffic Circle":
        return 5
    else:
        return 0
    
df['Crash_Location'] = df.apply(new_var, axis=1)
df['Crash_Location'].replace(" ", 99, inplace=True)


In [26]:
# Create a new column for Driver / Operator Type
df['Driver Type'] = df['Driver / Operator Type']

# Replace values in new column based on conditions
df['Driver Type'] = df['Driver Type'].replace(['None', 'In-Vehicle (Commercial / Test)', 'Consumer', 'Other, see Narrative', 'Unknown'], 0)
df['Driver Type'] = df['Driver Type'].replace('Remote (Commercial / Test)', 1)
df['Driver Type'] = df['Driver Type'].replace('In-Vehicle and Remote (Commercial / Test)', 2)

# Replace blanks with 99
df['Driver Type'] = df['Driver Type'].replace('', 99)


In [11]:
# Create a new column for the crash with type
df['Crash With Type'] = 0

# Assign values based on 'Crash With' column
df.loc[df['Crash With'].isin(['Passenger Car', 'SUV']), 'Crash With Type'] = 1
df.loc[df['Crash With'].isin(['Non-Motorist: Cyclist', 'Motorcycle', 'Non-Motorist: Pedestrian', 'Non-Motorist: Other']), 'Crash With Type'] = 2
df.loc[df['Crash With'] == 'Pickup Truck', 'Crash With Type'] = 3
df.loc[df['Crash With'] == 'Heavy Truck', 'Crash With Type'] = 4
df.loc[df['Crash With'].isin(['Other Fixed Object', 'Van', 'Other, see Narrative', 'First Responder Vehicle', 'Unknown', 'Animal', 'Bus', 'Pole / Tree']), 'Crash With Type'] = 5

# Replace blank values with 99
df['Crash With Type'].replace('', 99, inplace=True)


In [12]:
import numpy as np

# Replace blanks with 99
df['Narrative'].replace(" ", "99", inplace=True)

# Create new variable 'Disengaged' based on presence of keywords
df['Disengaged'] = np.where(df['Narrative'].str.contains('disengage|disengaged', case=False, na=False), 1, 0)


In [13]:
# Create a new variable for Roadway Surface
df['Roadway Surface Value'] = df['Roadway Surface'].map({'Dry': 0, 'Wet': 1, 'Snow / Slush / Ice': 2, 'Unknown': 2})

# Replace blank values with 99
df['Roadway Surface Value'] = df['Roadway Surface Value'].replace('', 99)

In [14]:
#Injury vs Non-injury Var
df['injuryVSnoninjury'] = df['Highest Injury Severity'].replace({'No Injuries Reported': 0,
                                                        'Minor': 1,
                                                        'Moderate': 1,
                                                        'Serious': 1,
                                                        'Unknown': 98})

In [33]:
# Keep only unique values of "Report ID" column in df
df = df.drop_duplicates(subset=["Report ID"], keep="first")


In [34]:
# Save the cleaned DataFrame to a new CSV file
df.to_csv('df_ADAS_pyC1.csv', index=False)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 910 entries, 0 to 1783
Columns: 142 entries, Report ID to Loongitude
dtypes: float64(7), int32(2), int64(7), object(126)
memory usage: 1009.5+ KB


In [18]:
from geopy.geocoders import GoogleV3

# create a geocoder object with your API key
geolocator = GoogleV3(api_key='YOUR_API_KEY')

# function to get latitude and longitude of a city
def get_lat_long(city):
    try:
        location = geolocator.geocode(city)
        return location.latitude, location.longitude
    except:
        return None, None

# add latitude and longitude columns to your dataframe
df['lat'], df['long'] = zip(*df['City'].apply(get_lat_long))

# replace blank cell with 99
df['City'].fillna(value=99, inplace=True)


In [30]:
df.head()

,Report ID,Report Version,Reporting Entity,Report Type,Report Month,Report Year,Report Submission Date,VIN,VIN - Unknown,Serial Number,...,Crash With Type,Disengaged,Roadway Surface Value,injuryVSnoninjury,latitude,longitude,lat,long,Laatitude,Loongitude
0,753-4922,3,"Kia America, Inc.",Monthly Update,3.0,2023.0,MAR-2023,5XYRKDLF9NG,,[MAY CONTAIN PERSONALLY IDENTIFIABLE INFORMATION],...,1,0,0.0,1,None,None,None,None,41.758926,0.964056
1,10039-5057,1,"BMW of North America, LLC",1-Day,NaN,NaN,MAR-2023,5UXJU4C03N9,,NaN,...,5,0,2.0,98,None,None,None,None,45.926777,-111.646626
2,12381-4852,3,"Daimler Trucks North America, LLC",Monthly Update,2.0,2023.0,MAR-2023,1FUJHHDR3NL,,[MAY CONTAIN PERSONALLY IDENTIFIABLE INFORMATION],...,4,0,0.0,0,None,None,None,None,33.898684,-81.275054
3,13781-5059,1,"Tesla, Inc.",1-Day,NaN,NaN,MAR-2023,5YJ3E1EA1LF,,NaN,...,5,0,2.0,98,None,None,None,None,37.429939,-122.253855
4,1243-4930,3,"Subaru of America, Inc.",Monthly Update,3.0,2023.0,MAR-2023,4S4BTGPD9M3,,NaN,...,4,0,2.0,0,None,None,None,None,36.420403,-81.473438


In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_app")

# Create a new column for latitude and longitude
df['Laaatitude'] = None
df['Looongitude'] = None

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    location = geolocator.geocode(str(row['City']) + ', ' + str(row['State']))
    if location is not None:
        df.at[index, 'Laaatitude'] = location.latitude
        df.at[index, 'Looongitude'] = location.longitude
